## A y B

In [22]:
using JuMP
using HiGHS

# Crear modelo con solver GLPK
model = Model()

# Datos millones de galones
capacidades = [6.0, 5.0, 8.0]
demandas = [4.0, 8.0, 7.0]

distancias = [
    120.0  180.0    NaN;
    300.0  100.0   80.0;
    200.0  250.0  120.0
]

costo_unitario = 0.10

0.1

In [23]:
@variable(model, x[1:3, 1:3] >= 0)

# (donde no hay conexión)
for i in 1:3, j in 1:3
    if isnan(distancias[i,j])
        fix(x[i,j], 0.0; force=true)
    end
end

# Función objetivo
@objective(model, Min, sum((distancias[i,j] * costo_unitario * 1000) * x[i,j]
        for i in 1:3, j in 1:3 if !isnan(distancias[i,j])))

# Restricciones de capacidad
for i in 1:3
    @constraint(model, sum(x[i,j] for j in 1:3) <= capacidades[i])
end

# Restricciones de demanda
for j in 1:3
    @constraint(model, sum(x[i,j] for i in 1:3) >= demandas[j])
end

In [24]:
set_optimizer(model, HiGHS.Optimizer)
optimize!(model)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 6 rows; 9 cols; 18 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [8e+03, 3e+04]
  Bound  [0e+00, 0e+00]
  RHS    [4e+00, 8e+00]
Presolving model
6 rows, 8 cols, 16 nonzeros  0s
6 rows, 8 cols, 16 nonzeros  0s
Presolve : Reductions: rows 6(-0); columns 8(-1); elements 16(-2)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(19) 0s
          6     2.4300000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 6
Objective value     :  2.4300000000e+05
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00


In [25]:
println("Estado: ", termination_status(model))
println("Costo total: \$", objective_value(model))

for i in 1:3, j in 1:3
    println("x[$i,$j] = ", value(x[i,j]), " millones de galones")
end


Estado: OPTIMAL
Costo total: $243000.0
x[1,1] = 4.0 millones de galones
x[1,2] = 2.0 millones de galones
x[1,3] = 0.0 millones de galones
x[2,1] = 0.0 millones de galones
x[2,2] = 5.0 millones de galones
x[2,3] = 0.0 millones de galones
x[3,1] = 0.0 millones de galones
x[3,2] = 1.0 millones de galones
x[3,3] = 7.0 millones de galones


## C

In [26]:
# Datos actualizados

model2 = Model()
capacidades = [6.0, 5.0, 8.0]
demandas = [4.0, 8.0, 4.0]

3-element Vector{Float64}:
 4.0
 8.0
 4.0

In [ ]:
# Costo de transporte por camión (por galón)
costo_excedente = [0.015, 0.022]

# Variables de decisión x[i,j]: cantidad enviada de refinería i a área j
@variable(model2, x[1:3, 1:3] >= 0)

# Variables para excedente enviado por camión desde refinerías 1 y 2
@variable(model2, e[1:2] >= 0)



2-element Vector{VariableRef}:
 e[1]
 e[2]

In [ ]:

# (donde no hay conexión)
for i in 1:3, j in 1:3
    if isnan(distancias[i,j])
        fix(x[i,j], 0.0; force=true)
    end
end

# Función objetivo
@objective(model2, Min,
    sum((distancias[i,j] * costo_unitario * 1000) * x[i,j] for i in 1:3, j in 1:3 if !isnan(distancias[i,j])) + 
    costo_excedente[1] * e[1] * 1_000_000 + costo_excedente[2] * e[2] * 1_000_000
)

# Restricciones de capacidad 
@constraint(model2, sum(x[1,j] for j in 1:3) + e[1] <= capacidades[1])
@constraint(model2, sum(x[2,j] for j in 1:3) + e[2] <= capacidades[2])
@constraint(model2, sum(x[3,j] for j in 1:3) <= capacidades[3])  # refinería 3 sin excedente externo

# Restricciones de demanda 
for j in 1:3
    @constraint(model2, sum(x[i,j] for i in 1:3) >= demandas[j])
end


In [29]:

set_optimizer(model2, HiGHS.Optimizer)
optimize!(model2)


Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 6 rows; 11 cols; 20 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [8e+03, 3e+04]
  Bound  [0e+00, 0e+00]
  RHS    [4e+00, 8e+00]
Presolving model
6 rows, 8 cols, 16 nonzeros  0s
6 rows, 8 cols, 16 nonzeros  0s
Presolve : Reductions: rows 6(-0); columns 8(-3); elements 16(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(16) 0s
          6     2.0700000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 6
Objective value     :  2.0700000000e+05
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00


In [ ]:

# Resultados
println("Estado: ", termination_status(model2))
println("Costo total: \$", objective_value(model2))

for i in 1:3, j in 1:3
    println("x[$i,$j] = ", value(x[i,j]), " millones de galones")
end

println("Excedente Refinería 1 (camión): ", value(e[1]), " millones de galones")
println("Excedente Refinería 2 (camión): ", value(e[2]), " millones de galones")

Estado: OPTIMAL
Costo total: $207000.0
x[1,1] = 4.0 millones de galones
x[1,2] = 2.0 millones de galones
x[1,3] = 0.0 millones de galones
x[2,1] = 0.0 millones de galones
x[2,2] = 5.0 millones de galones
x[2,3] = 0.0 millones de galones
x[3,1] = 0.0 millones de galones
x[3,2] = 1.0 millones de galones
x[3,3] = 4.0 millones de galones
Excedente Refinería 1 (camión): 0.0 millones de galones
Excedente Refinería 2 (camión): 0.0 millones de galones
